In [ ]:
import requests
import json
import datetime
import pytz
import time
from dateutil import parser
from requests.exceptions import RequestException

In [ ]:
def get_unix_timestamp(date_str):
    date_obj = parser.parse(date_str)
    day = date_obj.day
    month = date_obj.month
    year = date_obj.year
    hour = 15
    minute = 30
    tz = pytz.timezone('Asia/Kolkata')  # Indian Standard Time (IST)
    dt = tz.localize(datetime.datetime(year, month, day, hour, minute))
    return int(dt.timestamp())

In [ ]:
url_expiries = 'https://opstra.definedge.com/api/optionsimulator/simulatorexpiries'
headers = {'Cookie': 'JSESSIONID=37495F1EC678627FF4925324BE2686FD'}
reversed_expiries = requests.get(url_expiries, headers=headers)
expiries = reversed_expiries.json()[::-1]
expiry = ['31JAN2019']
expiries

In [ ]:
url_timestamps = 'https://opstra.definedge.com/api/optionsimulator/timestamps/2019-01-01&2023-05-25'
timestamps = requests.get(url_timestamps, headers=headers)
data_timestamp = timestamps.json()["timestamps"]
data_timestamp_dict = dict.fromkeys(data_timestamp, 0)

In [ ]:
data = dict.fromkeys(expiries, data_timestamp_dict)
# del data['31JAN2019'][1546314600]
data

In [ ]:
# for value in expiries:
#     print(value)
#     i = 0
#     while(get_unix_timestamp(value)>=data_timestamp[i]):
#         print(data_timestamp[i])
#         try:
#             url_data = 'https://opstra.definedge.com/api/optionsimulator/optionchain/'+str(data_timestamp[i])+'&NIFTY&'+value
#             data[value][data_timestamp[i]] = requests.get(url_data, headers=headers).json()
#         except RequestException:
#             data[value].pop(data_timestamp[i])
#             pass
#         i=i+1

In [ ]:
for value in expiries:
    print(value)
    maxts = get_unix_timestamp(value)
    unwanted = []
    for ts in data_timestamp:
        if(maxts>=ts):
            url_data = 'https://opstra.definedge.com/api/optionsimulator/optionchain/'+str(ts)+'&NIFTY&'+value
            try:
                r = requests.get(url_data, headers=headers)
                if r.status_code == 200:
                    data[value][ts] = r.json()
                else:
                    unwanted.append(ts)
            except RequestException as e:
                print(f"Request failed for timestamp {ts}: {e}")
                unwanted.append(ts)
#             r = requests.get(url_data, headers=headers)
#             if(r.status_code==200):
#                 data[value][ts] = r.json()
#             else:
#                 unwanted.append(ts)
        else:
            unwanted.append(ts)
    dict = data[value].copy()
    for val in unwanted:
        if val in dict:
            dict.pop(val)
    data[value] = dict
    dict.clear()

In [ ]:
filename = 'data.json'          #use the file extension .json
with open(filename, 'w') as file_object:  #open the file in write mode
 json.dump(data, file_object)

In [ ]:
1+2